In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
from mpl_toolkits.mplot3d import Axes3D
import scipy as sp
from multiprocessing import Pool

In [ ]:
N_realisation =0
np.random.seed(N_realisation)
particle_line_density = 20
N0_particle = 20 ** 3

# XP0 = np.random.uniform(-2, 2, N0_particle) 
# YP0 = np.random.uniform(-2, 2, N0_particle)
# ZP0 = np.random.uniform(-2, 2, N0_particle)

xp0 = np.linspace(-2, 2, particle_line_density)
yp0 = np.linspace(-2, 2, particle_line_density)
zp0 = np.linspace(-2, 2, particle_line_density)

XP0, YP0, ZP0 = np.meshgrid(xp0, yp0, zp0, indexing='ij')

St0 = 1
R0 = 1
a = R0
# a, alpha, R, Fr, gravity = 1, 1, 2/3, 5, False

xl, xr = -2, 2
yl, yr = -2, 2
zd, zu = -2, 2

dx_col, dy_col, dz_col = 1, 1, 1
Nx_col, Ny_col, Nz_col = 4, 4, 4
gridA_xc, gridA_yc, gridA_zc = np.meshgrid(np.linspace(xl+dx_col/2, xr-dx_col/2, Nx_col), 
                                           np.linspace(yl+dy_col/2, yr-dy_col, Ny_col), 
                                           np.linspace(zd+dz_col/2, zu-dz_col/2, Nz_col), indexing='ij')
gridB_xc, gridB_yc, gridB_zc = np.meshgrid(np.linspace(xl, xr, Nx_col+1), np.linspace(yl, yr, Ny_col+1), np.linspace(zd, zu, Nz_col+1), indexing='ij')

# Plot grid lines
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# # Plot lines along x-axis
# for i in range(Ny_col):
#     for j in range(Nz_col):
#         ax.plot(gridA_xc[:, i, j], gridA_yc[:, i, j], gridA_zc[:, i, j], color='r',linestyle='--')

# # Plot lines along y-axis
# for i in range(Nx_col):
#     for j in range(Nz_col):
#         ax.plot(gridA_xc[i, :, j], gridA_yc[i, :, j], gridA_zc[i, :, j], color='g',linestyle='--')


# # Plot lines along z-axis
# for i in range(Nx_col):
#     for j in range(Ny_col):
#         ax.plot(gridA_xc[i, j, :], gridA_yc[i, j, :], gridA_zc[i, j, :], color='b',linestyle='--')


ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

ax.scatter(XP0, YP0, ZP0,s=0.01, c='k', marker='o')


plt.show()

In [ ]:
X, Y, Z = np.meshgrid(np.linspace(-2, 2, 101), np.linspace(-2, 2, 100), np.linspace(-2, 2, 101), indexing='ij')
if Y.any() == 0:
    raise ValueError('Y cannot be zero')
PHI = np.arctan(Z/Y)
dist = np.sqrt(X**2 + Y**2 + Z**2)
r_planar = np.sqrt(Z**2 + Y**2)

mask = dist > R0

Ux = -(2 * (r_planar)**2 + X**2 -1) / 5
Ux[mask] = (-(a**5)/15) * (2/(a**3) + (r_planar[mask]**2 - 2*(X[mask]**2))/(dist[mask]**5))

Ur = X * r_planar / 5
Ur[mask] = (a**5) * X[mask] * r_planar[mask] / (5 * dist[mask]**5)

Uy = Ur * Y / r_planar
Uz = Ur * Z / r_planar

# a simple test to verify result
# test = Uy[:, :, 50]
# plt.contourf(X[:, :, 50], Y[:, :, 50], test, 50)
# plt.colorbar()

# # Plot velocity field
# ax = plt.figure().add_subplot(projection='3d')
# ax.quiver(X, Y, Z, Ux, Uy, Uz, length=0.2, normalize=True)


# plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
img = ax.scatter(X[dist<=R0], Y[dist<=R0], Z[dist<=R0], c=Uz[dist<=R0], s=0.1,  cmap='bwr')
fig.colorbar(img)
plt.xlabel('X')
plt.ylabel('Y')
plt.show()

In [ ]:
import advect_bubbles_3d as adv3D
import parallel_advect_bubbles_3d as pab3d

Bubbles_to_adv = np.hstack((np.arange(N0_particle)[:, np.newaxis], 
                            XP0.flatten()[:, np.newaxis], YP0.flatten()[:, np.newaxis], ZP0.flatten()[:, np.newaxis], 
                            np.zeros((N0_particle, 3)), St0 * np.ones((N0_particle, 1))))


updated_states = pab3d.main(initial_states = Bubbles_to_adv[:, 1:], t0=0.0, tf=150.0)

In [ ]:
import plot_bubbles

plot_bubbles.plot_bubbles(updated_states, R0)

In [ ]:
time = -1
inside = np.sqrt(x_out[:, time]**2 + y_out[:, time]**2 + z_out[:, time]**2) <= R0

# check = np.sqrt(x[inside, 0, time]**2 + out[inside, 1, time]**2 + out[inside, 2, time]**2)
avg_loc = np.mean(x_out[inside, time]), np.mean(y_out[inside, time]), np.mean(z_out[inside, time])
avg_loc